In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from pandas import DataFrame
from scipy import stats

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

In [2]:
## Reading files

train = pd.read_csv("../input/elo-merchant-category-recommendation/train.csv")
test = pd.read_csv("../input/elo-merchant-category-recommendation/test.csv")
# hist_txns = pd.read_csv("../input/historical_transactions.csv")
# new_txns = pd.read_csv("../input/new_merchant_transactions.csv")
# merchants = pd.read_csv("../input/elo-merchant-category-recommendation/merchants.csv")
all_txns = pd.read_csv("../input/transactions-data/all_txns.csv")
# all_txns = pd.concat([hist_txns, new_txns], ignore_index=True)
# all_txns.to_csv("all_txns.csv", index=False)

In [3]:
# print (all_txns.shape)

# merchants_subset = merchants[["merchant_id", "numerical_1", "numerical_2"]]

# txns_plus_merchants= pd.concat([all_txns, merchants_subset], axis=1)

# print (txns_plus_merchants.shape)
# print (txns_plus_merchants.columns)

In [4]:
# print (np.unique(all_txns["category_1"]))
# print (all_txns["category_1"].isna().sum())
# print (all_txns["category_2"].isna().sum())
# print (all_txns["category_3"].isna().sum())
# print (all_txns["category_3"].head(10))

# indexNames = all_txns[all_txns["category_3"].isna()].index
# all_txns.drop(indexNames, inplace=True)
# print (all_txns["category_3"].isna().sum())
# print (np.unique(all_txns["category_3"]))

# fill_na_with_mode = stats.mode(all_txns["category_3"])
# print (fill_na_with_mode)
# print (np.max(all_txns["category_2"]))
# print (np.min(all_txns["category_2"]))
# print (all_txns["category_2"].head(5))

In [5]:
# Missing values treatment

# print (train.shape)
# print (hist_txns.shape)
# print (new_txns.shape)
# print (all_txns.shape)
# print (all_txns.head(5))
# print (np.max(hist_txns["installments"]))
# print (np.min(hist_txns["installments"]))
# print (np.mean(hist_txns["installments"]))
# a = np.where(hist_txns["installments"] == 999)
# print (len(a[0]))

# indexNames = hist_txns[hist_txns["installments"] == 999].index
# hist_txns.drop(indexNames, inplace=True)
# print (np.max(hist_txns["installments"]))
# print (np.min(hist_txns["installments"]))
# print (np.mean(hist_txns["installments"]))
# b = np.where(hist_txns["installments"] == 999)
# print (len(b[0]))

In [6]:
txns_subset = all_txns[["card_id", "purchase_amount", "month_lag",
                        "installments", "authorized_flag", "category_1", "category_3"]]
# print (txns_subset.head(50))
grouped_txns = txns_subset.groupby("card_id").agg({
    "purchase_amount": [np.sum, np.mean],
    "month_lag": [np.sum, np.mean],
    "installments": [np.sum, np.mean],
    "authorized_flag": {"Y": lambda x: x[x == 'Y'].count(),
                        "N": lambda x: x[x == 'N'].count()},
    "category_1": {"Y": lambda x: x[x == 'Y'].count(),
                   "N": lambda x: x[x == 'N'].count()},
    "category_3": {"A": lambda x: x[x == 'A'].count(),
                   "B": lambda x: x[x == 'B'].count(),
                   "C": lambda x: x[x == 'C'].count()}
})
grouped_txns.columns = ["_".join(x) for x in grouped_txns.columns.ravel()]
grouped_txns = grouped_txns.reset_index()
# print (grouped_txns.shape)
print (grouped_txns.head(5))
# print (list(grouped_txns.columns))

/opt/conda/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


           card_id      ...       category_3_C
0  C_ID_00007093c1      ...                 24
1  C_ID_0001238066      ...                 36
2  C_ID_0001506ef0      ...                  0
3  C_ID_0001793786      ...                  0
4  C_ID_000183fdda      ...                 41

[5 rows x 14 columns]


In [7]:
train_data = train.join(grouped_txns.set_index("card_id"), on="card_id")
test_data = test.join(grouped_txns.set_index("card_id"), on="card_id")

# print (train_data.shape)
# print (test_data.shape)

In [8]:
# Extracting number of active months

train_data['first_active_month'] = pd.to_datetime(train_data['first_active_month'])
test_data['first_active_month'] = pd.to_datetime(test_data['first_active_month'])
current_date = datetime.datetime.today()

def _number_of_active_months(design_matrix):
    design_matrix['number_of_active_months'] = ((current_date - design_matrix['first_active_month']) / 30.).dt.days
    return design_matrix

train_mod = _number_of_active_months(train_data)
test_mod = _number_of_active_months(test_data)

## Imputing missing values in test data
fill_na_with_mean = np.mean(test_mod['number_of_active_months'])
test_mod['number_of_active_months'].fillna(fill_na_with_mean, inplace=True)

In [9]:
# Normalizing continuous features

features_to_norm = ['number_of_active_months', 'month_lag_sum', 'month_lag_mean',
                    'installments_sum', 'installments_mean', 'authorized_flag_Y',
                    'authorized_flag_N', 'category_1_Y', 'category_1_N',
                    'category_3_A', 'category_3_B', 'category_3_C']
for feature in features_to_norm:
    mean_value = np.mean(train_mod[feature])
    stdev = np.std(train_mod[feature])
    train_mod[feature] = train_mod[feature].apply(lambda x: (x - mean_value) / (stdev))
    test_mod[feature] = test_mod[feature].apply(lambda x: (x - mean_value) / (stdev))

In [10]:
# Creating dummies for categorical features

cat_features = ['feature_1', 'feature_2', 'feature_3']
response = ['target']

def _create_dummies_for_categorical_features(design_matrix):
    """Create dummies for categorical features"""
    design_matrix = pd.get_dummies(design_matrix, prefix=cat_features,
                                   columns=cat_features)
    return design_matrix

design_matrix_train = train_mod[list(test_mod.columns) + response]
design_matrix_test = test_mod[list(test_mod.columns)]

train_with_dummies = _create_dummies_for_categorical_features(design_matrix_train)
test_with_dummies = _create_dummies_for_categorical_features(design_matrix_test)

In [11]:
# Defining predictors for the model

predictors = ['purchase_amount_mean', 'category_1_Y', 'category_1_N',
              'month_lag_mean', 'installments_mean',
              'authorized_flag_N', 'authorized_flag_Y', 'number_of_active_months',
              'feature_2_1', 'feature_2_2', 'feature_2_3', 'feature_3_0', 'feature_3_1',
              'category_3_A', 'category_3_B', 'category_3_C',
              'feature_1_1', 'feature_1_2', 'feature_1_3', 'feature_1_4', 'feature_1_5']

# predictors = list(test_with_dummies.columns)
# predictors.remove('card_id')
# predictors.remove('first_active_month')
print (predictors)

['purchase_amount_mean', 'category_1_Y', 'category_1_N', 'month_lag_mean', 'installments_mean', 'authorized_flag_N', 'authorized_flag_Y', 'number_of_active_months', 'feature_2_1', 'feature_2_2', 'feature_2_3', 'feature_3_0', 'feature_3_1', 'category_3_A', 'category_3_B', 'category_3_C', 'feature_1_1', 'feature_1_2', 'feature_1_3', 'feature_1_4', 'feature_1_5']


In [12]:
# Model assessment using cross-validation

regressor = GradientBoostingRegressor()
parameters = {
              'loss': ['ls'],
              'learning_rate': [0.075],
              'n_estimators': [75],
              'subsample': [0.8],
              'criterion': ['friedman_mse'],
              'max_depth': [6],
              'min_samples_split': [2],
              'max_features': ['auto'],
              'warm_start': [True],
              'random_state': [9],
              'min_samples_leaf': [4],
              'verbose': [1],
              'n_iter_no_change': [20]
             }

model_grid = GridSearchCV(regressor,
                          param_grid=parameters,
                          scoring= "neg_mean_squared_error",
                          cv=5,
                          verbose=1,
                          n_jobs=5,
                          iid=False
                         )

model_grid.fit(train_with_dummies[predictors], train_with_dummies['target'])
print (model_grid.best_estimator_)
print ('Best model score: ', model_grid.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:  1.4min remaining:  2.1min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:  1.5min finished


      Iter       Train Loss      OOB Improve   Remaining Time 
         1          14.8000           0.0406            1.14m
         2          14.7256           0.0379            1.13m
         3          14.8202           0.0262            1.11m
         4          14.7068           0.0294            1.09m
         5          14.7860           0.0228            1.08m
         6          14.7343           0.0215            1.05m
         7          14.5320           0.0195            1.05m
         8          14.6440           0.0145            1.05m
         9          14.5452           0.0160            1.04m
        10          14.3007           0.0165            1.03m
        20          14.2559           0.0031           53.46s
        30          14.1883           0.0003           43.24s
        40          14.3069          -0.0018           32.82s
        50          14.0039          -0.0024           22.18s
        60          13.8321          -0.0041           12.90s
       

In [13]:
# Cross-validation results

cv_results = model_grid.cv_results_
results = DataFrame.from_dict(cv_results, orient='columns')
results.sort_values(['mean_test_score'], ascending=False, inplace=True)
# results.to_csv('GridSearch_results_4.csv', index=False)
print (results[['param_max_depth', 'param_min_samples_split',
                'param_min_samples_leaf', 'mean_test_score']])

  param_max_depth       ...       mean_test_score
0               6       ...            -14.470508

[1 rows x 4 columns]


/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

In [14]:
## Validation using best estimator

# best_model = model_grid.best_estimator_
# print ('Out of bag training score: ', best_model.oob_score_)
# validation_data['target_predicted'] = best_model.predict(validation_data[predictors])
# validation_data.to_csv('validation_data.csv', index=False)

# RMSE
mean_squared_error = (-1.) * model_grid.best_score_
rmse = np.sqrt(mean_squared_error)
print ('RMSE: ', rmse)

RMSE:  3.8040120958889365


In [15]:
# Fitting model with the best (chosen) estimator

best_model = model_grid.best_estimator_
model = best_model.fit(train_with_dummies[predictors], train_with_dummies['target'])
feature_imp1 = dict(zip(predictors, model.feature_importances_))
feature_imp2 = DataFrame.from_dict(feature_imp1, orient='index')
feature_imp2.sort_values([0], ascending=False, inplace=True)
print (feature_imp2.head(30))
# print ('Feature intercept: ', model.intercept_)

# Model score
score = best_model.score(train_with_dummies[predictors], train_with_dummies['target'])
print ()
print ('Model score: ', score)

# Predictions on test data
test_with_dummies['target'] = model.predict(test_with_dummies[predictors])
# print (test_with_dummies.head(10))

      Iter       Train Loss      OOB Improve   Remaining Time 
                                0
month_lag_mean           0.235734
number_of_active_months  0.154359
category_1_Y             0.119589
authorized_flag_N        0.104924
purchase_amount_mean     0.078704
installments_mean        0.075226
authorized_flag_Y        0.067788
category_1_N             0.052338
category_3_B             0.046087
category_3_C             0.023533
category_3_A             0.017775
feature_2_3              0.007680
feature_1_2              0.003335
feature_1_3              0.002432
feature_2_1              0.002012
feature_1_4              0.001838
feature_2_2              0.001638
feature_1_5              0.001410
feature_3_1              0.001239
feature_1_1              0.001203
feature_3_0              0.001156

Model score:  0.06339098739279514


In [16]:
# Submissions

submission_file = test_with_dummies[['card_id', 'target']]
# print (submission_file.head(20))

submission_file.to_csv('submission_22_' + str(rmse) + '.csv', index=False)